In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# A Logic Puzzle

The following exercise is taken from the book 
<a href="https://www.amazon.de/Logeleien-Zweistein-ihren-Antworten-Wegner/dp/B006YF0VUE">"99 Logeleien von Zweistein"</a>.
This book has been published 1968.  It is written by 
<a href="http://de.wikipedia.org/wiki/Thomas_von_Randow">Thomas von Randow</a>.

---
The gentlemen Amann, Bemann, Cemann and Demann are called - not necessarily in the same order - by their first names Erich, Fritz, Gustav and Heiner. They are all married to exactly one woman. We also know the following about them and their wives:

- Either Amann's first name is Heiner, or Bemann's wife is Inge.
- If Cemann is married to Josefa, then - **and only in this case** - Klara's husband is **not** called Fritz.
- If Josefa's husband is **not** called Erich, then Inge is married to Fritz.
- If Luise's husband is called Fritz, then Klara's husband's first name is **not** Gustav.
- If the wife of Fritz is called Inge, then Erich is **not** married to Josefa.
- If Fritz is **not** married to Luise, then Gustav's wife's name is Klara.
- Either Demann is married to Luise, or Cemann is called Gustav.

*What are the full fullnames of these gentlemen, and what are their wives' first names?*

---

We are going to solve this problem by coding it in propositional logic and we will solve the resulting set of clauses using the Davis-Putnam algorithm.  In order to code the problem, we will use the following propositional variables:

- $\texttt{MaleName<}x\texttt{,}z\texttt{>}$ for any male first name $x$ and any surname $z$ expresses
  that the gentleman with first name $x$ has surname $z$.
- $\texttt{Married<}x\texttt{,}y\texttt{>}$ for any male first name $x$ and any female first name $y$ expresses
  that the gentleman with first name $x$ is married to the lady with first name $y$.
- $\texttt{FemaleName<}y\texttt{,}z\texttt{>}$ for any female first name $y$ and any last name $z$ expresses
  that the lady with first name $y$ has surname $z$.

We are using the symbols $\texttt{<}$ and $\texttt{>}$ as part of the propositional variables because we want to show the structure of these variables and the parser for propositional logic accepts these symbols as part of propositional variables.

In [2]:
FirstMale   = { "Erich",  "Fritz", "Gustav", "Heiner" }
FirstFemale = { "Inge",  "Josefa", "Klara",  "Luise"  }
SurNames    = { "Amann", "Bemann", "Cemann", "Demann" }

In [3]:
%%capture
%run 07-Davis-Putnam-JW.ipynb

In [4]:
%%capture
%run Propositional-Logic-Parser.ipynb

In [5]:
%%capture
%run 04-CNF.ipynb

The function $\texttt{makeVar}(f, x, y)$ creates a propositional variable of the form $\texttt{f<}x\texttt{,}y\texttt{>}$.

In [6]:
def makeVar(f, x, y):
    return f + '<' + x + ',' + y + '>'

In [7]:
makeVar('Married', 'Heiner', 'Klara')

'Married<Heiner,Klara>'

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ computes a set of clauses expressing the fact that at most one of the variables of $S$ is <tt>True</tt>.

In [8]:
def atMostOne(S): 
    return { frozenset({('¬',p), ('¬', q)}) for p in S
                                            for q in S 
                                            if  p != q 
           }

Given a set of propositional variables $S$, the function $\texttt{atLeastOne}(S)$ computes a set of clauses expressing the fact that at least one of the variables of $S$ is <tt>True</tt>.

In [9]:
def atLeastOne(S):
    return { frozenset(S) }

$S$ is a set of propositional variables. The expression $\texttt{exactlyOne}(S)$ creates a set of clauses.  This set expresses the fact that exactly one of the variables in the set $S$ is true.

In [10]:
def exactlyOne(S):
    return atMostOne(S) | atLeastOne(S)

In [11]:
exactlyOne({ 'a', 'b', 'c' })

{frozenset({('¬', 'a'), ('¬', 'b')}),
 frozenset({'a', 'b', 'c'}),
 frozenset({('¬', 'a'), ('¬', 'c')}),
 frozenset({('¬', 'b'), ('¬', 'c')})}

For two sets $A$ and $B$ that have the same number of elements and a function symbol $f$, the procedure $\texttt{bijective}(A, B)$ computes a set of clauses that is equivalent to the formula
$$   \bigl(\forall x \in A: \exists! y \in B: f\langle x, y\rangle\bigr) \wedge
     \bigl(\forall y \in B: \exists! x \in A: f\langle x, y\rangle\bigr)
$$
Here the expression $f\langle x,y\rangle$ is the name of a propositional variable and the expression $\exists!x:p(x)$ is to be read as "There exists exactly one $x$ such that $p(x)$ holds".

In [12]:
from itertools import combinations

def bijective(A, B, f):
    Clauses = set()
    
    for a in A:
        # All 
        Clauses.add(frozenset(makeVar(f, a, b) for b in B))
        for b1, b2 in combinations(B, 2):
            Clauses.add(frozenset({('¬', makeVar(f, a, b1)), ('¬', makeVar(f, a, b2))}))
    for b in B:
        Clauses.add(frozenset(makeVar(f, a, b) for a in A))
        for a1, a2 in combinations(A, 2):
            Clauses.add(frozenset({('¬', makeVar(f, a1, b)), ('¬', makeVar(f, a2, b))}))
    return Clauses

In [14]:
bijective({'a', 'b', 'c', 'd'}, {'x', 'y', 'z', 'u'}, 'f')

{frozenset({('¬', 'f<a,y>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<b,y>')}),
 frozenset({('¬', 'f<a,z>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<b,x>'), ('¬', 'f<c,x>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<c,u>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<d,u>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,y>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,x>')}),
 frozenset({('¬', 'f<b,y>'), ('¬', 'f<d,y>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<d,u>')}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<c,y>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<b,y>')}),
 frozenset({('¬', 'f<b,z>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<b,x>')}),
 frozenset({'f<a,y>', 'f<b,y>', 'f<c,y>', 'f<d,y>'}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<d,x>')}),
 frozenset({'f<a,z>', 'f<b,z>', 'f<c,z>', 'f<d,z>'}),
 frozenset({('¬', 'f<b,u>'),

For example, the function call `bijective({'a', 'b', 'c', 'd'}, {'x', 'y', 'z', 'u'}, 'f')` returns the following set of clauses:
    
```
{frozenset({('¬', 'f<a,x>'), ('¬', 'f<a,y>')}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<d,x>')}),
 frozenset({'f<a,y>', 'f<b,y>', 'f<c,y>', 'f<d,y>'}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<d,u>')}),
 frozenset({('¬', 'f<c,u>'), ('¬', 'f<c,y>')}),
 frozenset({('¬', 'f<c,u>'), ('¬', 'f<d,u>')}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<c,y>')}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<b,z>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,x>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<c,x>')}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<d,u>')}),
 frozenset({'f<d,u>', 'f<d,x>', 'f<d,y>', 'f<d,z>'}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<c,u>')}),
 frozenset({('¬', 'f<b,y>'), ('¬', 'f<b,z>')}),
 frozenset({('¬', 'f<b,y>'), ('¬', 'f<c,y>')}),
 frozenset({('¬', 'f<b,y>'), ('¬', 'f<d,y>')}),
 frozenset({'f<a,z>', 'f<b,z>', 'f<c,z>', 'f<d,z>'}),
 frozenset({('¬', 'f<a,z>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<b,x>'), ('¬', 'f<d,x>')}),
 frozenset({('¬', 'f<c,x>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<d,u>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<d,y>'), ('¬', 'f<d,z>')}),
 frozenset({'f<c,u>', 'f<c,x>', 'f<c,y>', 'f<c,z>'}),
 frozenset({('¬', 'f<b,z>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<a,z>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<c,u>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<b,x>'), ('¬', 'f<b,z>')}),
 frozenset({'f<a,u>', 'f<a,x>', 'f<a,y>', 'f<a,z>'}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<b,u>')}),
 frozenset({'f<b,u>', 'f<b,x>', 'f<b,y>', 'f<b,z>'}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<b,x>')}),
 frozenset({('¬', 'f<c,z>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<a,z>'), ('¬', 'f<b,z>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,y>')}),
 frozenset({('¬', 'f<d,u>'), ('¬', 'f<d,y>')}),
 frozenset({('¬', 'f<c,y>'), ('¬', 'f<d,y>')}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<b,x>')}),
 frozenset({('¬', 'f<b,z>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<d,u>'), ('¬', 'f<d,x>')}),
 frozenset({('¬', 'f<d,x>'), ('¬', 'f<d,y>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<a,u>'), ('¬', 'f<a,z>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<d,x>')}),
 frozenset({('¬', 'f<c,u>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<d,y>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<c,y>')}),
 frozenset({('¬', 'f<b,x>'), ('¬', 'f<c,x>')}),
 frozenset({('¬', 'f<d,x>'), ('¬', 'f<d,z>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<b,y>')}),
 frozenset({('¬', 'f<c,y>'), ('¬', 'f<c,z>')}),
 frozenset({('¬', 'f<c,u>'), ('¬', 'f<c,x>')}),
 frozenset({'f<a,u>', 'f<b,u>', 'f<c,u>', 'f<d,u>'}),
 frozenset({('¬', 'f<b,x>'), ('¬', 'f<b,y>')}),
 frozenset({('¬', 'f<b,u>'), ('¬', 'f<b,y>')}),
 frozenset({'f<a,x>', 'f<b,x>', 'f<c,x>', 'f<d,x>'})}
```

The function $\texttt{parseAndNormalize}(s)$ takes a string $s$, parses this string as a propositional formula and then turns this formula into a set of clauses.

In [15]:
def parseAndNormalize(s):
    nestedTuple = parse(s)
    Clauses     = normalize(nestedTuple)
    return Clauses

The function `exclusiveOr(a, b)` computes the *exclusive or* of the formulas $a$ and $b$, which are given as strings. The resulting formula itself is converted into CNF.

In [16]:
def exclusiveOr(a, b):
    formula = f'(({a}) ↔ ¬({b}))'
    return parseAndNormalize(formula)

In [17]:
exclusiveOr('p', 'q')

{frozenset({'p', 'q'}), frozenset({('¬', 'p'), ('¬', 'q')})}

Given a set of male first names `FirstMale`, a set of female first names `FirstFemale`, and a set of surnames `Surnames`,
the function `consistentNames(FirstMale, FirstFemale, Surnames)` returns a set of clauses that ensures that if the man
`x` has the surname `z` and the woman `y` also has the surname `z`, then `x` and `y` have to be married.

In [18]:
def consistentNames(FirstMale, FirstFemale, SurNames):
    Clauses = set()
    for x in FirstMale:
        for y in FirstFemale:
            for z in SurNames:
                Clauses |= parseAndNormalize(f'MaleName<{x},{z}> ∧ FemaleName<{y},{z}> → Married<{x},{y}>')
    return Clauses

In [19]:
consistentNames(FirstMale, FirstFemale, SurNames)

{frozenset({('¬', 'FemaleName<Klara,Bemann>'),
            ('¬', 'MaleName<Erich,Bemann>'),
            'Married<Erich,Klara>'}),
 frozenset({('¬', 'FemaleName<Klara,Bemann>'),
            ('¬', 'MaleName<Fritz,Bemann>'),
            'Married<Fritz,Klara>'}),
 frozenset({('¬', 'FemaleName<Josefa,Amann>'),
            ('¬', 'MaleName<Heiner,Amann>'),
            'Married<Heiner,Josefa>'}),
 frozenset({('¬', 'FemaleName<Luise,Cemann>'),
            ('¬', 'MaleName<Heiner,Cemann>'),
            'Married<Heiner,Luise>'}),
 frozenset({('¬', 'FemaleName<Josefa,Demann>'),
            ('¬', 'MaleName<Heiner,Demann>'),
            'Married<Heiner,Josefa>'}),
 frozenset({('¬', 'FemaleName<Luise,Demann>'),
            ('¬', 'MaleName<Erich,Demann>'),
            'Married<Erich,Luise>'}),
 frozenset({('¬', 'FemaleName<Luise,Amann>'),
            ('¬', 'MaleName<Gustav,Amann>'),
            'Married<Gustav,Luise>'}),
 frozenset({('¬', 'FemaleName<Luise,Demann>'),
            ('¬', 'MaleName<Fritz,D

Below, you might need the following symbols: ¬, ∧, ∨, →, ↔

In [20]:
def computeClauses(FirstMale, FirstFemale, SurNames):
    # Jedem männlichen Vornamen ist genau ein Nachname zugeordnet und umgekehrt.
    Clauses  = bijective(FirstMale, SurNames, "MaleName")
    # Jeder Mann ist mit genau einer Frau verheiratet und umgekehrt.
    Clauses |= bijective(FirstMale, FirstFemale, "Married")
    # Jede Frau hat genau einen Nachnamen
    Clauses |= bijective(FirstFemale, SurNames, "FemaleName")
    # Die Namen sind konsistent.
    Clauses |= consistentNames(FirstMale, FirstFemale, SurNames)
    # Entweder ist Amanns Vorname Heiner, oder Bemanns Frau heisst Inge.
    Clauses |= exclusiveOr('Amann ∧ Heiner', 'Bemann ∧ Inge')
    # Wenn Cemann mit Josefa verheiratet ist, dann – und nur in diesem Falle –
    # heisst Klaras Mann nicht Fritz.
    Clauses |= parseAndNormalize("Cemann ∧ Josefa ↔ Klara ∧ ¬Fritz")
    # Wenn Josefas Mann nicht Erich heisst, dann ist Inge mit Fritz verheiratet.
    Clauses |= parseAndNormalize("Josefa ∧ ¬Erich → Inge ∧ Fritz")
    # Wenn Luises Mann Fritz heisst, dann ist der Vorname von Klaras Mann nicht Gustav.
    Clauses |= parseAndNormalize("Luise ∧ Fritz  → Klara ∧ ¬Gustav")
    # Wenn die Frau von Fritz Inge heisst, dann ist Erich nicht mit Josefa verheiratet.
    Clauses |= parseAndNormalize("Fritz ∧ Inge → Erich ∧ ¬Josefa")
    # Wenn Fritz nicht mit Luise verheiratet ist, dann heisst Gustavs Frau Klara.
    Clauses |= parseAndNormalize("Fritz ∧ ¬Luise  → Gustav ∧ Klara")
    # Entweder ist Demann mit Luise verheiratet, oder Cemann heisst Gustav.
    Clauses |= exclusiveOr('Demann ∧ Luise', 'Cemann ∧ Gustav')
    return Clauses

In [21]:
Clauses = computeClauses(FirstMale, FirstFemale, SurNames)
Clauses

{frozenset({('¬', 'FemaleName<Klara,Bemann>'),
            ('¬', 'MaleName<Erich,Bemann>'),
            'Married<Erich,Klara>'}),
 frozenset({('¬', 'FemaleName<Inge,Demann>'),
            ('¬', 'FemaleName<Klara,Demann>')}),
 frozenset({('¬', 'FemaleName<Luise,Amann>'),
            ('¬', 'FemaleName<Luise,Demann>')}),
 frozenset({'MaleName<Erich,Amann>',
            'MaleName<Fritz,Amann>',
            'MaleName<Gustav,Amann>',
            'MaleName<Heiner,Amann>'}),
 frozenset({('¬', 'MaleName<Fritz,Bemann>'), ('¬', 'MaleName<Fritz,Cemann>')}),
 frozenset({'Married<Erich,Luise>',
            'Married<Fritz,Luise>',
            'Married<Gustav,Luise>',
            'Married<Heiner,Luise>'}),
 frozenset({('¬', 'FemaleName<Inge,Bemann>'),
            ('¬', 'FemaleName<Inge,Demann>')}),
 frozenset({('¬', 'MaleName<Erich,Demann>'),
            ('¬', 'MaleName<Gustav,Demann>')}),
 frozenset({('¬', 'FemaleName<Klara,Bemann>'),
            ('¬', 'MaleName<Fritz,Bemann>'),
            'Married<

There are 242 different clauses.

In [22]:
len(Clauses)

254

In [29]:
def compute_solution(FirstMale, FirstFemale, SurNames):
    Clauses = computeClauses(FirstMale, FirstFemale, SurNames)
    Result  = solve(Clauses)
    return Result

In [30]:
%%time
Solution = compute_solution(FirstMale, FirstFemale, SurNames)

CPU times: user 38.5 ms, sys: 0 ns, total: 38.5 ms
Wall time: 38.3 ms


In [31]:
def arb(S):
    for x in S: 
        return x

In [32]:
def onlyPositive(Solution):
    Result = set()
    for Clause in Solution:
        literal = arb(Clause)
        if isinstance(literal, str):
            Result.add(literal)
    return Result

In [33]:
onlyPositive(Solution)

{'Bemann',
 'Cemann',
 'FemaleName<Inge,Cemann>',
 'FemaleName<Josefa,Demann>',
 'FemaleName<Klara,Bemann>',
 'FemaleName<Luise,Amann>',
 'Gustav',
 'Inge',
 'MaleName<Erich,Amann>',
 'MaleName<Fritz,Cemann>',
 'MaleName<Gustav,Bemann>',
 'MaleName<Heiner,Demann>',
 'Married<Erich,Luise>',
 'Married<Fritz,Inge>',
 'Married<Gustav,Klara>',
 'Married<Heiner,Josefa>'}

In [34]:
import re

def extractFirst(s):
        m = re.search('<([A-Za-z]+),', s)
        return m.group(1)

def extractSecond(s):
        m = re.search(',([A-Za-z]+)>', s)
        return m.group(1)

In [35]:
def displaySolution(Solution):
    Married = {}
    Names   = {}
    for Unit in Solution:
        for l in Unit:
            if isinstance(l, str):
                if l[:7] == "Married":
                    x = extractFirst(l)
                    y = extractSecond(l)
                    Married[x] = y
                elif l[:8] == "MaleName":
                    x = extractFirst(l)
                    y = extractSecond(l)
                    Names[x] = y
    for x in Married:
        print(f"{x} {Names[x]} is married to {Married[x]}.")

In [36]:
displaySolution(Solution)

Gustav Bemann is married to Klara.
Erich Amann is married to Luise.
Heiner Demann is married to Josefa.
Fritz Cemann is married to Inge.


## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [37]:
def negateSolution(UnitClauses):
    return { complement(arb(unit)) for unit in UnitClauses }

In [38]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

{('¬', 'a'), 'b'}

In [39]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = solve(Clauses)
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [40]:
%%time
checkUniqueness(Solution, Clauses)

ERROR: The solution is not unique!
CPU times: user 50.8 ms, sys: 0 ns, total: 50.8 ms
Wall time: 50.7 ms
